In [19]:
# this file is for device on the client side

# load the dataset

# This file is for trainning
# Run this on the server, or as we called offline. 

import argparse
import base64
import cv2
import datetime
from Models import gatedmodel,mobilenetv2, resnet
import numpy as np
import os
import PIL
import psutil
import sys
import time
import torch
from tqdm import tqdm
from Utils import utils, encoder

middle_sizes_mobile = [1,2,4,8,16]
middle_sizes_resnet = [1,2,4,8,16,32]

middle_sizes = {'mobilenet': middle_sizes_mobile, 'resnet': middle_sizes_resnet}
reduced_sizes = {'cifar-10': (32,32), 'imagenet': (224,224)}
reduced_rates = {'mobilenet': 2, 'resnet': 4}

dataset = 'cifar-10'
model = 'mobilenet'
i_stop = 100

width, height = reduced_sizes[dataset][0]/reduced_rates[model], \
                reduced_sizes[dataset][1]/reduced_rates[model]
middle_size = middle_sizes[model]

# client include client, middle and gate

client = mobilenetv2.mobilenetv2_splitter_client(num_classes = 10, weight_root='./Weights/'+dataset+'/', device='cpu')
# client = resnet.resnet_splitter_client(num_classes=1000, weight_root='./Weights/'+dataset+'/', device='cpu', layers=50)

middle_models = []
for i in range(len(middle_size)):
    middle_models.append(mobilenetv2.MobileNetV2_middle(middle=middle_size[i]))
    # middle_models.append(resnet.resnet_middle(middle=middle_size[i]))

gate_models = []
for i in range(len(middle_size)):
    gate_models.append(gatedmodel.ExitGate(in_planes=middle_size[i],
                                           height = height, width=width))

# eval
client.eval()
for i in range(len(middle_size)):
    middle_models[i].eval()
    gate_models[i].eval()

# quantize
client = torch.ao.quantization.quantize_dynamic(client, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
for i in range(len(middle_size)):
    middle_models[i] = torch.ao.quantization.quantize_dynamic(middle_models[i], {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
    gate_models[i] = torch.ao.quantization.quantize_dynamic(gate_models[i], {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)

# 2. dataset
# directly read bmp image from the storage
data_root = '../data/'+dataset+'-client/'
data_client_out = []
for i in range(len(middle_size)):
    data_client_out.append('../data/'+dataset+'-'+model+'-client-'+str(middle_size[i])+'/')
    if not os.path.exists(data_client_out[i]):
        os.makedirs(data_client_out[i])
    
images_list = os.listdir(data_root)
images_list.remove('labels.txt')
# remove ending with jpg
images_list = [x for x in images_list if x.endswith('.bmp')]
images_list = sorted(images_list)

client_time = [0] * len(middle_size)

# this is test the overspeed, so we don't need to load the models
with torch.no_grad():
        for i, i_path in tqdm(enumerate(images_list)):
            if i >= i_stop:
                break
            
            image_path = data_root + i_path
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = image.astype(np.float32)/255.0
            image = torch.tensor(image)
            image = image.unsqueeze(0)
            image = image.permute(0, 3, 1, 2)
            
            with torch.no_grad():
                s_time = time.time()
                client_out = client(image).detach()
                for j in range(len(middle_size)):
                    middle_in = middle_models[j].in_layer(client_out)
                    gate_out = gate_models[j](middle_in)
                    middle_in = middle_in.squeeze(0)
                    middle_int = utils.float_to_uint(middle_in)
                    middle_int = middle_int.numpy().copy(order='C')
                    middle_int = middle_int.astype(np.uint8)
                    send_in = base64.b64encode(middle_int)
                    # store it in the folder
                    with open(data_client_out[j] + i_path[:-4], 'wb') as f:
                        f.write(send_in)
                    s1_time = time.time()
                    client_time[j] += s1_time - s_time

for i in range(len(client_time)):
    client_time[i] /= i_stop / 1000 # ms per frame

# print the list without [ and ]
out_string = str(client_time).replace('[','').replace(']','')
print(dataset, model)
print(out_string)


100it [00:00, 226.90it/s]

cifar-10 mobilenet
0.8240985870361328, 1.9571852684020996, 2.882969379425049, 3.5321617126464844, 4.309632778167725


In [22]:
# read a image and store it
# image = cv2.imread('../data/imagenet-client/0.bmp', cv2.IMREAD_COLOR)
# # encode
# image = image.astype(np.uint8)
# image = base64.b64encode(image)
# # store
# with open('../data/imagenet-client/0b', 'wb') as f:
#     f.write(image)

In [28]:
# data path
data_root = '../data/imagenet-client/'
# get '.bmp'
import os 
images_list = os.listdir(data_root)
images_list = [x for x in images_list if x.endswith('.bmp')]

data_out = '../data/imagenet-jpeg/'

import cv2
# compress the image jpeg
for i_path in images_list:
    image_path = data_root + i_path
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    for quality in range(5, 100, 5):
        cv2.imwrite(data_out+i_path[:-4]+'_'+str(quality)+'.jpg', image, [int(cv2.IMWRITE_JPEG_QUALITY), quality])


In [6]:
# load embbeded 
dataset = 'cifar-10'
model = 'mobilenet'
out_path = ['../data/'+dataset+'-jpeg25/', '../data/'+dataset+'-jpeg75/', '../data/'+dataset+'-cjpeg/']
import torch
from Models import mobilenetv2, resnet
from torchvision import transforms
import time
import base64
import numpy as np
import cv2
for opath in out_path:

    
    model = mobilenetv2.MobileNetV2(num_classes=10)
    model.load_state_dict(torch.load('./Weights/'+dataset+'/pretrained/mobilenetv2.pth'))
    model = model.to('cuda')
    model.eval()

    root_path = '../data/'+dataset+'-client/'
    labels = root_path+'labels.txt'
    labels = open(labels, 'r')
    labels = labels.readlines()
    labels = [x[:-1] for x in labels]
    labels = torch.tensor([int(x) for x in labels])
    labels

    images = [str(x) for x in range(600)]
    preds = []

    
    if dataset == 'cifar-10':
        normal = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])
    elif dataset == 'imagenet':
        normal = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

    with torch.no_grad():
        server_time = []
        for image in images:
            
            image_path = opath + image
            i_read = open(image_path, 'rb')
            image = i_read.read()
            # decode
            s_time = time.time()
            image = base64.b64decode(image)
            image = np.frombuffer(image, dtype=np.uint8)
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            image = normal(image)
            image = image.unsqueeze(0)
            image = image.to('cuda')
            pred = model(image)
            pred = torch.argmax(pred, dim=1)
            preds.append(pred.item())
            e_time = time.time()
            server_time.append(e_time - s_time)

    preds = torch.tensor(preds)
    accuracy = [0] * 10 
    server_time_box = [0] * 10
    # calculate the accuracy every 60 frames
    for i in range(0, 600, 60):
        accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60
        server_time_box[i//60] = sum(server_time[i:i+60]) / 60*1000

    print(opath)
    print(accuracy)
    print(server_time_box)

../data/cifar-10-jpeg25/
[0.65, 0.7, 0.6333333333333333, 0.6833333333333333, 0.65, 0.7, 0.7166666666666667, 0.6666666666666666, 0.7666666666666667, 0.7166666666666667]
[3.321786721547445, 3.167839845021566, 3.113889694213867, 3.452531496683757, 3.3099611600240073, 3.1742294629414873, 3.095245361328125, 3.180229663848877, 3.098241488138835, 3.1754453976949053]
../data/cifar-10-jpeg75/
[0.8, 0.8666666666666667, 0.7666666666666667, 0.7833333333333333, 0.7166666666666667, 0.7666666666666667, 0.8333333333333334, 0.7666666666666667, 0.9, 0.8833333333333333]
[3.245274225870768, 3.1121174494425454, 3.1221787134806314, 3.11356782913208, 3.1094511349995932, 3.129263718922933, 3.1371434529622397, 3.1410296758015948, 3.1263311703999834, 3.1348228454589844]
../data/cifar-10-cjpeg/
[0.8166666666666667, 0.8, 0.7666666666666667, 0.8166666666666667, 0.7166666666666667, 0.8, 0.8333333333333334, 0.7833333333333333, 0.8666666666666667, 0.8666666666666667]
[3.190298875172933, 3.1914353370666504, 3.16008726

In [13]:
# load embbeded 
dataset = 'cifar-10'
model = 'mobilenet'

out_path = ['../data/'+dataset+'-jpeg25-ML/', '../data/'+dataset+'-jpeg75-ML/', '../data/'+dataset+'-cjpeg-ML/']
import torch
from Models import mobilenetv2, resnet
from torchvision import transforms
import time
import base64
import numpy as np
import cv2
for opath in out_path:

    model = mobilenetv2.MobileNetV2(num_classes=10)
    model.load_state_dict(torch.load('./Weights/'+dataset+'/pretrained/mobilenetv2.pth'))
    model = model.to('cuda')
    model.eval()

    root_path = '../data/'+dataset+'-client/'
    labels = root_path+'labels.txt'
    labels = open(labels, 'r')
    labels = labels.readlines()
    labels = [x[:-1] for x in labels]
    labels = torch.tensor([int(x) for x in labels])
    labels

    images = [str(x) for x in range(600)]
    preds = []

    from torchvision import transforms
    if dataset == 'cifar-10':
        normal = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])
    elif dataset == 'imagenet':
        normal = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

    with torch.no_grad():
        server_time = []
        for image in images:
            
            image_path = opath + image
            # check is the path exist
            i_read = open(image_path, 'rb')
            image = i_read.read()
            
            # decode
            s_time = time.time()
            image = base64.b64decode(image)
            image = np.frombuffer(image, dtype=np.uint8)
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            image = normal(image)
            image = image.unsqueeze(0)
            image = image.to('cuda')
            pred = model(image)
            pred = torch.argmax(pred, dim=1)
            preds.append(pred.item())
            e_time = time.time()
            server_time.append(e_time - s_time)

    preds = torch.tensor(preds)
    accuracy = [0] * 10 
    server_time_box = [0] * 10
    # calculate the accuracy every 60 frames
    for i in range(0, 600, 60):
        accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60
        server_time_box[i//60] = sum(server_time[i:i+60]) / 60*1000

    print(opath)
    # change it to 4 float 
    for i in range(10):
        accuracy[i] = float(str('%.4f'%accuracy[i]))
        server_time_box[i] = float(str('%.4f'%server_time_box[i]))
    print(accuracy)
    print(server_time_box)

../data/cifar-10-jpeg25-ML/
[0.7333, 0.7333, 0.5667, 0.55, 0.6, 0.65, 0.7167, 0.6, 0.6833, 0.7167]
[3.1158, 3.1078, 3.2164, 3.2086, 3.0422, 2.9906, 2.9844, 3.0055, 2.9925, 2.9831]
../data/cifar-10-jpeg75-ML/
[0.8167, 0.8167, 0.7333, 0.8167, 0.6667, 0.75, 0.75, 0.6833, 0.8167, 0.8833]
[3.0094, 2.9996, 3.0027, 3.0042, 2.996, 2.9956, 3.0027, 2.9946, 3.0037, 3.0019]
../data/cifar-10-cjpeg-ML/
[0.7333, 0.85, 0.7167, 0.7, 0.7, 0.8167, 0.7, 0.75, 0.7667, 0.8333]
[3.0358, 3.0187, 3.0196, 3.0213, 3.0209, 3.0359, 3.0335, 3.0569, 3.0195, 3.0334]


In [64]:
# load embbeded 
dataset = 'imagenet-20'
model = 'resnet'

out_path = ['../data/'+dataset+'-jpeg25-ML/', '../data/'+dataset+'-jpeg75-ML/', '../data/'+dataset+'-cjpeg-ML/']
import torch
from Models import mobilenetv2, resnet
from torchvision import transforms
import time
import base64
import numpy as np
import cv2
for opath in out_path:

    model = resnet.resnet50(num_classes=1000)
    model.load_state_dict(torch.load('./Weights/'+'imagenet'+'/pretrained/resnet50.pth'))
    model = model.to('cuda')
    model.eval()

    root_path = '../data/'+dataset+'-client/'
    labels = root_path+'labels.txt'
    labels = open(labels, 'r')
    labels = labels.readlines()
    labels = [x[:-1] for x in labels]
    labels = torch.tensor([int(x) for x in labels])
    labels

    images = [str(x) for x in range(600)]
    preds = []

    from torchvision import transforms
    if dataset == 'cifar-10':
        normal = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])
    elif dataset == 'imagenet-20':
        normal = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

    with torch.no_grad():
        server_time = []
        for image in images:
            
            image_path = opath + image
            # check is the path exist
            i_read = open(image_path, 'rb')
            image = i_read.read()
            
            # decode
            s_time = time.time()
            image = base64.b64decode(image)
            image = np.frombuffer(image, dtype=np.uint8)
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            image = normal(image)
            image = image.unsqueeze(0)
            image = image.to('cuda')
            pred = model(image)
            pred = torch.argmax(pred, dim=1)
            preds.append(pred.item())
            e_time = time.time()
            server_time.append(e_time - s_time)

    preds = torch.tensor(preds)
    accuracy = [0] * 10 
    server_time_box = [0] * 10
    # calculate the accuracy every 60 frames
    for i in range(0, 600, 60):
        accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60
        server_time_box[i//60] = sum(server_time[i:i+60]) / 60*1000

    print(opath)
    # change it to 4 float 
    for i in range(10):
        accuracy[i] = float(str('%.4f'%accuracy[i]))
        server_time_box[i] = float(str('%.4f'%server_time_box[i]))
    print(accuracy)
    print(server_time_box)

../data/imagenet-20-jpeg25-ML/
[0.45, 0.4333, 0.4833, 0.3833, 0.4167, 0.45, 0.4, 0.5833, 0.55, 0.65]
[5.3645, 4.9753, 5.3577, 5.3949, 5.6408, 5.4833, 5.6144, 5.8528, 5.0103, 5.2712]
../data/imagenet-20-jpeg75-ML/
[0.6667, 0.5833, 0.7167, 0.5833, 0.75, 0.7, 0.6833, 0.7, 0.75, 0.75]
[5.7225, 5.2585, 5.5746, 5.5671, 5.8161, 5.5767, 5.8245, 6.0693, 5.3879, 5.6762]
../data/imagenet-20-cjpeg-ML/
[0.5833, 0.5, 0.6333, 0.5333, 0.6333, 0.6333, 0.6167, 0.6833, 0.6667, 0.7333]
[7.1993, 6.7135, 7.0339, 7.1668, 7.3736, 7.011, 7.3398, 7.8842, 6.6181, 7.098]


In [66]:
# load embbeded 
dataset = 'imagenet-20'
model = 'resnet'

out_path = ['../data/'+dataset+'-jpeg25/', '../data/'+dataset+'-jpeg75/', '../data/'+dataset+'-cjpeg/']
import torch
from Models import mobilenetv2, resnet
from torchvision import transforms
import time
import base64
import numpy as np
import cv2
for opath in out_path:

    model = resnet.resnet50(num_classes=1000)
    model.load_state_dict(torch.load('./Weights/'+'imagenet'+'/pretrained/resnet50.pth'))
    model = model.to('cuda')
    model.eval()

    root_path = '../data/'+dataset+'-client/'
    labels = root_path+'labels.txt'
    labels = open(labels, 'r')
    labels = labels.readlines()
    labels = [x[:-1] for x in labels]
    labels = torch.tensor([int(x) for x in labels])
    labels

    images = [str(x) for x in range(600)]
    preds = []

    from torchvision import transforms
    if dataset == 'cifar-10':
        normal = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])
    elif dataset == 'imagenet-20':
        normal = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

    with torch.no_grad():
        server_time = []
        for image in images:
            
            image_path = opath + image
            # check is the path exist
            i_read = open(image_path, 'rb')
            image = i_read.read()
            
            # decode
            s_time = time.time()
            image = base64.b64decode(image)
            image = np.frombuffer(image, dtype=np.uint8)
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            image = normal(image)
            image = image.unsqueeze(0)
            image = image.to('cuda')
            pred = model(image)
            pred = torch.argmax(pred, dim=1)
            preds.append(pred.item())
            e_time = time.time()
            server_time.append(e_time - s_time)

    preds = torch.tensor(preds)
    accuracy = [0] * 10 
    server_time_box = [0] * 10
    # calculate the accuracy every 60 frames
    for i in range(0, 600, 60):
        accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60
        server_time_box[i//60] = sum(server_time[i:i+60]) / 60*1000

    print(opath)
    # change it to 4 float 
    for i in range(10):
        accuracy[i] = float(str('%.4f'%accuracy[i]))
        server_time_box[i] = float(str('%.4f'%server_time_box[i]))
    print(accuracy)
    print(server_time_box)

../data/imagenet-20-jpeg25/
[0.7, 0.6667, 0.7333, 0.6667, 0.7333, 0.7333, 0.7667, 0.7167, 0.75, 0.8167]
[5.312, 4.9395, 5.2042, 5.2532, 5.6307, 5.4263, 5.544, 5.6886, 5.0294, 5.3301]
../data/imagenet-20-jpeg75/
[0.7333, 0.7333, 0.75, 0.7833, 0.7833, 0.85, 0.9, 0.8, 0.8, 0.8833]
[5.7784, 5.3921, 5.6901, 5.6746, 5.697, 5.4954, 5.7028, 5.9074, 5.7537, 5.5621]
../data/imagenet-20-cjpeg/
[0.7667, 0.7667, 0.7833, 0.8, 0.8667, 0.8333, 0.9167, 0.8167, 0.7833, 0.85]
[7.1461, 6.7674, 7.1299, 7.2767, 7.5804, 7.4747, 7.6563, 8.4955, 7.1995, 7.4197]


In [6]:
# load embbeded 
dataset = 'cifar-10'
model = 'mobile'

out_path = ['../data/cifar-10-mobile-gate-emb/']

import torch
from Models import mobilenetv2, resnet
from torchvision import transforms
import time
import base64
import numpy as np
import cv2
import math

for opath in out_path:

    _, server = mobilenetv2.mobilenetv2_splitter(num_classes=10, weight_root='./Weights/'+dataset+'/')
    server = server.to('cuda')
    server.eval()

    root_path = '../data/'+'cifar-10'+'-client/'
    labels = root_path+'labels.txt'
    labels = open(labels, 'r')
    labels = labels.readlines()
    labels = [x[:-1] for x in labels]
    labels = torch.tensor([int(x) for x in labels])

    images = [str(x) for x in range(600)]
    preds = []

    middle_sizes = [1,2,4,8,16]
    middles = []
    for i in range(len(middle_sizes)):
        middles.append(mobilenetv2.MobileNetV2_middle(middle=middle_sizes[i]))
        middles[i].load_state_dict(torch.load('./Weights/'+dataset+'/middle/mobile_cifar-10_middle_'+str(middle_sizes[i])+'.pth'))
        middles[i] = middles[i].to('cuda')
        middles[i].eval()

    with torch.no_grad():
        server_time = []
        for image in images:
            
            image_path = opath + image
            # check is the path exist
            i_read = open(image_path, 'rb')
            img = i_read.read()
            
            # decode
            s_time = time.time()
            img = base64.b64decode(img)
            img = np.frombuffer(img, dtype=np.uint8)
            image_len = len(img)
            image_c = int(image_len/16/16)
            img = img.reshape(image_c, 16, 16)
            img = torch.tensor(img)
            img = img.unsqueeze(0)
            img = img.to('cuda')
            img = img.float()/255.0

            f2 = open(opath + image + '_helper', 'r')
            f2_line = f2.read()
            f2.close()

            image_max, image_min = f2_line.split(',')
            image_max = float(image_max)
            image_min = float(image_min)

            img = img * (image_max - image_min) + image_min

            middle = middles[int(math.log2(image_c))]
            img = middle.out_layer(img)
            pred = server(img)
            pred = torch.argmax(pred, dim=1)    
            preds.append(pred.item())
            e_time = time.time()
            server_time.append(e_time - s_time)

    preds = torch.tensor(preds)
    accuracy = [0] * 10 
    server_time_box = [0] * 10
    # calculate the accuracy every 60 frames
    for i in range(0, 600, 60):
        accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60
        server_time_box[i//60] = sum(server_time[i:i+60]) / 60*1000

    print(opath)
    # change it to 4 float 
    for i in range(10):
        accuracy[i] = float(str('%.4f'%accuracy[i]))
        server_time_box[i] = float(str('%.4f'%server_time_box[i]))
    print(accuracy)
    print(server_time_box)

../data/cifar-10-mobile-gate-emb/
[0.8333, 0.8667, 0.7833, 0.8333, 0.8, 0.8167, 0.85, 0.8167, 0.85, 0.9]
[5.2649, 3.0527, 3.2189, 3.5748, 3.0986, 3.2618, 3.383, 3.2862, 3.0711, 3.1564]


In [57]:
# load embbeded 
dataset = 'imagenet'
model = 'reset'

out_path = ['../data/imagenet-resnet-gate-emb/']

import torch
from Models import mobilenetv2, resnet
from torchvision import transforms
import time
import base64
import numpy as np
import cv2
import math
from Utils import utils
import sys

for opath in out_path:

    _, server = resnet.resnet_splitter(num_classes=1000, weight_root='./Weights/'+dataset+'/', layers=50)
    server = server.to('cuda')
    server.eval()

    root_path = '../data/'+'imagenet-20'+'-client/'
    labels = root_path+'labels.txt'
    labels = open(labels, 'r')
    labels = labels.readlines()
    labels = [x[:-1] for x in labels]
    labels = torch.tensor([int(x) for x in labels])

    images = [str(x) for x in range(600)]
    preds = []

    middle_sizes = [1,2,4,8,16,32]
    middles = []
    for i in range(len(middle_sizes)):
        middles.append(resnet.resnet_middle(middle=middle_sizes[i]))
        middles[i].load_state_dict(torch.load('./Weights/'+dataset+'/middle/resnet_imagenet_middle_'+str(middle_sizes[i])+'.pth'))
        middles[i] = middles[i].to('cuda')
        middles[i].eval()

    with torch.no_grad():
        server_time = []
        for image in images:

            f2 = open(opath + image + '_helper', 'r')
            f2_line = f2.read()
            f2.close()
            image_max, image_min = f2_line.split(',')
            image_min, image_max = float(image_min), float(image_max)

            image_path = opath + image
            # check is the path exist
            i_read = open(image_path, 'rb')
            img = i_read.read()
            
            # decode
            s_time = time.time()
            img = base64.b64decode(img)
            img = np.frombuffer(img, dtype=np.uint8)
            image_len = len(img)
            image_c = int(image_len/56/56)
            img = np.reshape(img, (image_c, 56, 56))
            
            img = torch.tensor(img)
            img = img.unsqueeze(0)
            img = utils.uint_to_float(img)
            img = utils.renormalize(img, image_min, image_max)
            img = img.to('cuda')

            c_gate = int(math.log2(image_c))
            if c_gate < len(middle_sizes):
                img = middles[c_gate].out_layer(img)
            pred = server(img)
            pred = torch.argmax(pred, dim=1)    
            preds.append(pred.item())
            e_time = time.time()
            server_time.append(e_time - s_time)

    preds = torch.tensor(preds)
    accuracy = [0] * 10 
    server_time_box = [0] * 10
    # calculate the accuracy every 60 frames
    for i in range(0, 600, 60):
        accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60
        server_time_box[i//60] = sum(server_time[i:i+60]) / 60*1000

    print(opath)
    # change it to 4 float 
    for i in range(10):
        accuracy[i] = float(str('%.4f'%accuracy[i]))
        server_time_box[i] = float(str('%.4f'%server_time_box[i]))
    print(accuracy)
    print(server_time_box)

../data/imagenet-resnet-gate-emb/
[0.7667, 0.7167, 0.8667, 0.7333, 0.7333, 0.75, 0.8667, 0.7833, 0.8, 0.85]
[3.4139, 3.3888, 3.402, 3.477, 3.4073, 3.3982, 3.3837, 3.3911, 3.3917, 3.4164]


In [45]:
# plot ground truth

from Dataloaders import dataloader_cifar10, dataloader_image_20
from Models import resnet, mobilenetv2
import matplotlib.pyplot as plt
import torch

# cifar-10
_, test_c, _ = dataloader_cifar10.Dataloader_cifar10_val(test_batch=1)

client_m, server_m = mobilenetv2.mobilenetv2_splitter(num_classes=10, weight_root='./Weights/cifar-10/')

_, test_i, _, _ = dataloader_image_20.Dataloader_imagenet_20_integrated(test_batch=1)

client_r, server_r = resnet.resnet_splitter(num_classes=1000, weight_root='./Weights/imagenet/', layers=50)

client_m = client_m.to('cuda')
client_m.eval()
client_r = client_r.to('cuda')
client_r.eval()
server_m = server_m.to('cuda')
server_m.eval()
server_r = server_r.to('cuda')
server_r.eval()

accuracy_c = [0] * 600
accuracy_i = [0] * 600

acc_c_box = [0] * 10
acc_i_box = [0] * 10

middle = resnet.resnet_middle(middle=8)
middle.load_state_dict(torch.load('./Weights/imagenet/middle/resnet_imagenet_middle_8.pth'))
middle = middle.to('cuda')
middle.eval()

with torch.no_grad():
    for i, (data, target) in enumerate(test_c):
        break
        if i >= 600:
            break
        data = data.to('cuda')
        target = target.to('cuda')
        client_out = client_m(data)
        server_out = server_m(client_out)
        pred = torch.argmax(server_out, dim=1)
        accuracy_c[i] = torch.sum(pred == target).item() / 1
    for i, (data, target, _) in enumerate(test_i):
        if i >= 600:
            break
        data = data.to('cuda')
        target = target.to('cuda')
        client_out = client_r(data)
        middle_in = middle.in_layer(client_out)
        client_out, mmin, mmax = utils.normalize_return(middle_in)
        client_out = utils.float_to_uint(client_out)
        client_out = utils.uint_to_float(client_out)
        client_out = utils.renormalize(client_out, mmin, mmax)
        client_out = middle.out_layer(client_out)
        print(client_out[0,0,0])
        sys.exit()
        server_out = server_r(client_out)
        pred = torch.argmax(server_out, dim=1)
        accuracy_i[i] = torch.sum(pred == target).item() / 1
    
for i in range(0, 600, 60):
    acc_c_box[i//60] = sum(accuracy_c[i:i+60]) / 60
    acc_i_box[i//60] = sum(accuracy_i[i:i+60]) / 60

print('acc_c_box ', acc_c_box)
print('acc_i_box ', acc_i_box)

Files already downloaded and verified
Files already downloaded and verified
tensor([1.2153, 0.4002, 0.6014, 1.0841, 1.0084, 0.6028, 0.4152, 0.3421, 0.4330,
        0.7573, 0.9349, 0.8802, 0.7422, 0.3174, 0.4431, 0.7466, 0.9605, 1.0331,
        0.8091, 1.0024, 0.9166, 0.9664, 0.9782, 1.0465, 1.0376, 0.6554, 0.4028,
        0.5763, 0.7455, 0.6971, 0.6846, 0.8152, 0.7967, 0.9449, 0.8786, 0.6162,
        0.5315, 0.6142, 0.7118, 0.6386, 0.6240, 0.6276, 0.6622, 0.6761, 0.6565,
        0.5542, 0.6210, 0.7828, 0.7554, 0.7438, 0.8716, 0.7254, 0.5950, 0.3314,
        0.3848, 0.8445], device='cuda:0')


NameError: name 'sys' is not defined